In [2]:
# importing the required modules
import requests
import pandas as pd
from sqlalchemy import create_engine 

# requests --> To make http requests.
# pandas --> To store and Transform the data in the form of DataFrames
# sqlalchemy.create_engine --> To create a connection and to load data into targeted location.

In [19]:
def extract()-> dict:
    """ This is a extract function to get data from
    http://universities.hipolabs.com """
    
    API_URL = "http://universities.hipolabs.com/search?country=United+States"
    data = requests.get(API_URL).json()
    return data

In [20]:
def transform(data:dict)->pd.DataFrame:
    """
    Transforms the data and build a dataset as per required format
    """
    df = pd.DataFrame(data)
    print(f"Total number of universities from API {len(data)}")
    df = df[df['name'].str.contains('Texas')]
    print(f"Number of universities in Texas {len(df)}")
    df['domain'] = [','.join(map(str,l)) for l in df['domains']]
    df['web_page'] = [','.join(map(str,l)) for l in df['web_pages']]
    df = df.reset_index(drop=True)
    return df[['domains','country','web_page','name']]


In [21]:
def load(df:pd.DataFrame)->None:
    """
    This is the function to load data frame df into the target database 
    """
    connection_string = "postgresql://<username>:<password>@<host_address>/<database>"
    db = create_engine(connection_string)
    conn = db.connect()
    df.to_sql('texas_uni',con=conn,if_exists='replace',index=False)
    print("Process_completed")
    

In [22]:
data = extract()
df = transform(data)
load(df)

Total number of universities from API 2300
Number of universities in Texas 41
Process_completed
